## Librerie

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report

from keras import models
from keras import layers
from keras import optimizers

## Import Dati

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Assignment 1/train.csv')

In [0]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Assignment 1/test.csv')

In [0]:
train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Overview del Dataset

In [0]:
train.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,2.700000e+04,27000.000000,27000.000000,27000.000000,27000.000000,2.700000e+04,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000
mean,167196.518519,1.603185,1.852444,1.551074,35.466667,-0.017704,-0.131259,-0.164963,-0.218704,-0.265148,-0.289963,51323.763407,49209.214889,4.698653e+04,43268.021926,40306.842667,38901.908259,5689.029370,5.931783e+03,5224.739037,4852.083778,4795.497259,5221.980185,0.221222
std,129337.261882,0.489246,0.789157,0.522311,9.206436,1.125608,1.197965,1.198714,1.171908,1.136924,1.153581,73528.597002,71167.111334,6.929217e+04,64392.383099,60803.533634,59510.024370,16911.464096,2.352149e+04,17786.906985,15978.287986,15043.584614,18069.890373,0.415078
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3600.000000,2986.000000,2.663000e+03,2313.000000,1746.000000,1261.000000,1000.000000,8.345000e+02,390.000000,291.750000,248.750000,106.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22431.000000,21188.500000,2.008800e+04,19067.500000,18118.500000,17137.500000,2100.000000,2.008500e+03,1800.000000,1500.000000,1504.500000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67591.750000,64145.250000,6.022025e+04,54656.750000,50245.500000,49326.250000,5005.000000,5.000000e+03,4500.000000,4006.000000,4036.250000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000


In [0]:
pandas_profiling.ProfileReport(train)

Number of variables,24
Number of observations,27000
Total Missing (%),0.0%
Total size in memory,4.9 MiB
Average record size in memory,192.0 B
Numeric,17
Categorical,0
Boolean,2
Date,0
Text (Unique),0
Rejected,5


In [0]:
print("la classe target contiene " + str((train["default.payment.next.month"]==0).sum())+ " zeri")
print("la classe target contiene " + str((train["default.payment.next.month"]==1).sum()) + " uni")
a = len(train["default.payment.next.month"])
print("In percentuale il valore di 1 rispetto all'intera classe vale " + str((train["default.payment.next.month"]==1).sum()/a))

la classe target contiene 21027 zeri
la classe target contiene 5973 uni
In percentuale il valore di 1 rispetto all'intera classe vale 0.2212222222222222


In [0]:
train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Preprocessing

Si analizzano le variabili EDUCATION, MARRIAGE, AGE, LIMIT_BAL e tutte le variabili BILL_AMT e PAY_AMT

Variabile AGE

In [0]:
print(train["AGE"].unique())

[24 26 34 37 57 29 23 28 35 51 41 30 49 39 40 27 47 33 32 54 22 31 25 46
 42 43 45 56 44 53 38 63 36 52 48 55 60 50 75 58 73 59 21 67 61 66 62 70
 72 64 65 71 69 68 79 74]


In [0]:
temp =  pd.get_dummies(pd.cut(train["AGE"], [0, 20,30,40,50,70, float("inf")]))
temp.columns = ["0-20", "21-30", "31-40", "41-50", "51-70", "71+"]
train = train.join(temp)
train.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', '0-20', '21-30', '31-40', '41-50',
       '51-70', '71+'],
      dtype='object')

In [0]:
train = train.drop(columns="AGE")

Variabile EDUCATION

In [0]:
print(train["EDUCATION"].unique())

[2 1 3 5 4 6 0]


In [0]:
train["EDUCATION"].value_counts()

2    12623
1     9532
3     4428
5      254
4      108
6       42
0       13
Name: EDUCATION, dtype: int64

In [0]:
# Si decide di accorpare tutti i valori superiori a 3 in quanto sono titoli di studio diversi dai più comuni e quindi rari, saranno indicati con 0
train["EDUCATION"][train["EDUCATION"]>3] =0
train["EDUCATION"].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2    12623
1     9532
3     4428
0      417
Name: EDUCATION, dtype: int64

In [0]:
temp =  pd.get_dummies(pd.cut(train["EDUCATION"], [0, 1, 2, 3, 4]))
temp.columns = ["0", "1", "2", "3"]
train = train.join(temp)
train.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3',
       'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
       'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
       'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', '0-20', '21-30', '31-40', '41-50',
       '51-70', '71+', '0', '1', '2', '3'],
      dtype='object')

In [0]:
train = train.drop(columns="EDUCATION")

Variabile MARRIAGE

In [0]:
print(train["MARRIAGE"].unique())

[1 2 3 0]


In [0]:
temp =  pd.get_dummies(pd.cut(train["MARRIAGE"], [0, 1, 2, 3,4]))
temp.columns = ["SPOSATO0", "SPOSATO1", "SPOSATO2", "SPOSATO3"]
train = train.join(temp)
train.columns

Index(['LIMIT_BAL', 'SEX', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4',
       'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default.payment.next.month',
       '0-20', '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3'],
      dtype='object')

In [0]:
train = train.drop(columns="MARRIAGE")

Variabile LIMIT_BAL

In [0]:
print(train["LIMIT_BAL"].unique())

[  20000.  120000.   90000.   50000.  500000.  100000.  140000.  200000.
  260000.  630000.   70000.  250000.  320000.  180000.  130000.  450000.
   60000.  230000.  280000.  360000.   10000.  210000.  150000.  380000.
  310000.  400000.   80000.  290000.  340000.  300000.   30000.  240000.
  470000.  160000.  480000.  350000.  330000.  110000.  420000.  170000.
  370000.  270000.  220000.   40000.  190000.  510000.  460000.  440000.
  410000.  490000.  390000.  580000.  600000.  610000.  700000.  670000.
  430000.  550000. 1000000.  530000.  710000.  560000.  520000.  750000.
  680000.  640000.   16000.  570000.  590000.  660000.  620000.  720000.
  800000.  540000.  760000.  690000.  650000.  780000.  730000.  740000.]


In [0]:
len(train["LIMIT_BAL"].unique())

80

In [0]:
a = train["LIMIT_BAL"].unique()
a.sort()
a

array([  10000.,   16000.,   20000.,   30000.,   40000.,   50000.,
         60000.,   70000.,   80000.,   90000.,  100000.,  110000.,
        120000.,  130000.,  140000.,  150000.,  160000.,  170000.,
        180000.,  190000.,  200000.,  210000.,  220000.,  230000.,
        240000.,  250000.,  260000.,  270000.,  280000.,  290000.,
        300000.,  310000.,  320000.,  330000.,  340000.,  350000.,
        360000.,  370000.,  380000.,  390000.,  400000.,  410000.,
        420000.,  430000.,  440000.,  450000.,  460000.,  470000.,
        480000.,  490000.,  500000.,  510000.,  520000.,  530000.,
        540000.,  550000.,  560000.,  570000.,  580000.,  590000.,
        600000.,  610000.,  620000.,  630000.,  640000.,  650000.,
        660000.,  670000.,  680000.,  690000.,  700000.,  710000.,
        720000.,  730000.,  740000.,  750000.,  760000.,  780000.,
        800000., 1000000.])

In [0]:
temp =  pd.get_dummies(pd.cut(train["LIMIT_BAL"], [0, 80000, 150000, 230000, 310000, 390000, 470000, 550000, 630000, 710000, 1000001]))
temp.columns = ["LIMITE_1", "LIMITE_2", "LIMITE_3", "LIMITE_4", "LIMITE_5", "LIMITE_6", "LIMITE_7", "LIMITE_8", "LIMITE_9", "LIMITE_10"]
train = train.join(temp)
train.columns

Index(['LIMIT_BAL', 'SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default.payment.next.month',
       '0-20', '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3', 'LIMITE_1', 'LIMITE_2',
       'LIMITE_3', 'LIMITE_4', 'LIMITE_5', 'LIMITE_6', 'LIMITE_7', 'LIMITE_8',
       'LIMITE_9', 'LIMITE_10'],
      dtype='object')

In [0]:
train = train.drop(columns="LIMIT_BAL")

Dal grafico della correlazione prodotto da pandas_profiling si nota che esiste una elevata correlazione tra le variabili BILL_AMT: sono variabili relative alle spese del conto e si decide di applicare prima una trasformazione logaritmica a ciascuna, poi si calcola una media di esse per generare "AVG_BILL_AMT".

In [0]:
#Tutti i valori negativi sono convertiti in zero per potervi applicare il logaritmo (non applicabile altrimenti se non 
#con l'ottenimento di numeri complessi)
for i in range(1,7):
    train["BILL_AMT" + str(i)][train["BILL_AMT" + str(i)] < 0] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
#Si applica la trasformazione logaritmica log(num + 1)
for i in range(1,7):
    train["BILL_AMT" + str(i)] = np.log(train["BILL_AMT" + str(i)] + 1)
    train["PAY_AMT" + str(i)] = np.log(train["PAY_AMT" + str(i)] + 1)

In [0]:
train["AVG_BILL_AMT"] = train[["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"]].mean(axis = 1)
train = train.drop(columns=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

## Dataset intermedio

In [0]:
train.columns

Index(['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', '0-20', '21-30', '31-40', '41-50',
       '51-70', '71+', '0', '1', '2', '3', 'SPOSATO0', 'SPOSATO1', 'SPOSATO2',
       'SPOSATO3', 'LIMITE_1', 'LIMITE_2', 'LIMITE_3', 'LIMITE_4', 'LIMITE_5',
       'LIMITE_6', 'LIMITE_7', 'LIMITE_8', 'LIMITE_9', 'LIMITE_10',
       'AVG_BILL_AMT'],
      dtype='object')

Riordinamento delle colonne

In [0]:
train = train[['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20', '21-30', '31-40', '41-50',
       '51-70', '71+', '0', '1', '2', '3', 'SPOSATO0', 'SPOSATO1', 'SPOSATO2',
       'SPOSATO3', 'LIMITE_1', 'LIMITE_2', 'LIMITE_3', 'LIMITE_4', 'LIMITE_5',
       'LIMITE_6', 'LIMITE_7', 'LIMITE_8', 'LIMITE_9', 'LIMITE_10',
       'AVG_BILL_AMT', 'default.payment.next.month']]

In [0]:
train.head()

,SEX,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,0-20,21-30,31-40,41-50,51-70,71+,0,1,2,3,SPOSATO0,SPOSATO1,SPOSATO2,SPOSATO3,LIMITE_1,LIMITE_2,LIMITE_3,LIMITE_4,LIMITE_5,LIMITE_6,LIMITE_7,LIMITE_8,LIMITE_9,LIMITE_10,AVG_BILL_AMT,default.payment.next.month
0,2,2,2,-1,-1,-2,-2,0.000000,6.536692,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,3.808189,1
1,2,-1,2,0,0,0,2,0.000000,6.908755,6.908755,6.908755,0.000000,7.601402,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,7.929061,1
2,2,0,0,0,0,0,0,7.325808,7.313887,6.908755,6.908755,6.908755,8.517393,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,9.696908,0
3,2,0,0,0,0,0,0,7.601402,7.610853,7.090910,7.003974,6.975414,6.908755,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,10.527606,0
4,1,-1,0,-1,0,0,0,7.601402,10.510041,9.210440,9.105091,6.536692,6.522093,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,9.643320,0


## Split dei dati

In [0]:
datitrain, datitest = train_test_split(train, test_size = 0.2, stratify = train["default.payment.next.month"], random_state = 123)

In [0]:
datitrain["default.payment.next.month"].value_counts()

0    16822
1     4778
Name: default.payment.next.month, dtype: int64

## Downsampling

Come già osservato in precedenza anche in questa porzione ridotta dei dati (ottenuta tramite sampling stratificato) si nota la presenza di una forte sproporzione tra elementi etichettati come 0 e 1 della classe target. <br>Si procede con una fase di Downsampling per evitare di incombere in condizioni di Overfitting o Generalizzazione, ed in generale per far si che le performance del modello siano migliori.

In [0]:
#Creazione di due dataset, ciascuno contenente osservazioni etichettate solo con 1 o con 0
df0 = datitrain[datitrain["default.payment.next.month"]==0]
df1 = datitrain[datitrain["default.payment.next.month"]==1]

In [0]:
#Si effettua una riduzione dei dati della classe maggioritaria (0) prelevando senza ripescaggio un numero di osservazioni uguale alla classe minoritaria (1)
downdf0 = resample(df0, replace=False, n_samples=datitrain["default.payment.next.month"].value_counts()[1], random_state=123)

In [0]:
#Si combinano entrambi i dataset per generare quello bilanciato
temp = pd.concat([downdf0, df1])

In [0]:
#Conteggio delle nuove classi
temp["default.payment.next.month"].value_counts()

1    4778
0    4778
Name: default.payment.next.month, dtype: int64

In [0]:
temp = temp.sample(frac=1).reset_index(drop=True)

In [0]:
ncol = len(temp.columns)-1

Ora i dati risultano bilanciati, si procede con l'aggiornamento dei valori di X_train e Y_train come richiesto dalla rete neurale

In [0]:
temp.columns

Index(['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20',
       '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3', 'LIMITE_1', 'LIMITE_2',
       'LIMITE_3', 'LIMITE_4', 'LIMITE_5', 'LIMITE_6', 'LIMITE_7', 'LIMITE_8',
       'LIMITE_9', 'LIMITE_10', 'AVG_BILL_AMT', 'default.payment.next.month'],
      dtype='object')

## Preparazione dei dati per la Rete Neurale

In [0]:
X_train, y_train = temp[['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20',
       '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3', 'LIMITE_1', 'LIMITE_2',
       'LIMITE_3', 'LIMITE_4', 'LIMITE_5', 'LIMITE_6', 'LIMITE_7', 'LIMITE_8',
       'LIMITE_9', 'LIMITE_10', 'AVG_BILL_AMT']], temp['default.payment.next.month']

X_test, y_test = datitest[['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20',
       '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3', 'LIMITE_1', 'LIMITE_2',
       'LIMITE_3', 'LIMITE_4', 'LIMITE_5', 'LIMITE_6', 'LIMITE_7', 'LIMITE_8',
       'LIMITE_9', 'LIMITE_10', 'AVG_BILL_AMT']], datitest["default.payment.next.month"]


Di seguito si trasformano le strutture dati da dare in input:

In [0]:
X_train = X_train.as_matrix()
y_train = y_train.values.reshape(9556,1)

X_test = X_test.as_matrix()
y_test = y_test.values.reshape(5400,1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


## Misure di performance

Si definiscono le misure recall_m, precision_m e f1_m; Quest'ultima verrà utilizzata per poter effettuare la valutazione del modello (inserita all'interno di metrics nella fase model.compile)

In [0]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Rete Neurale

Prima versione:

model = models.Sequential()

model.add(layers.Dense(32, input_dim = ncol, activation = "relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

Seconda versione:

model = models.Sequential()

model.add(layers.Dense(32, input_dim = ncol, activation = "relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

In [0]:
model = models.Sequential()

model.add(layers.Dense(32, input_dim = ncol, activation = "relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(1, activation="sigmoid"))




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                1248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [0]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc", f1_m])

In [0]:
history = model.fit(X_train, y_train, 
                    batch_size=128,
                    epochs=50,
                    validation_split=0.3)

Train on 6689 samples, validate on 2867 samples
Epoch 1/50
6689/6689 [==============================] - 2s 341us/step - loss: 0.8840 - acc: 0.5219 - f1_m: 0.5184 - val_loss: 0.6662 - val_acc: 0.6652 - val_f1_m: 0.6521
Epoch 2/50
6689/6689 [==============================] - 0s 22us/step - loss: 0.6962 - acc: 0.5521 - f1_m: 0.5371 - val_loss: 0.6586 - val_acc: 0.6641 - val_f1_m: 0.6047
Epoch 3/50
6689/6689 [==============================] - 0s 21us/step - loss: 0.6800 - acc: 0.5754 - f1_m: 0.5497 - val_loss: 0.6499 - val_acc: 0.6512 - val_f1_m: 0.5571
Epoch 4/50
6689/6689 [==============================] - 0s 21us/step - loss: 0.6578 - acc: 0.6091 - f1_m: 0.5739 - val_loss: 0.6317 - val_acc: 0.6582 - val_f1_m: 0.5634
Epoch 5/50
6689/6689 [==============================] - 0s 21us/step - loss: 0.6490 - acc: 0.6186 - f1_m: 0.5725 - val_loss: 0.6235 - val_acc: 0.6718 - val_f1_m: 0.6019
Epoch 6/50
6689/6689 [==============================] - 0s 21us/step - loss: 0.6340 - acc: 0.6411 - f1_m: 

In [0]:
x_plot = list(range(1,history.epoch[-1]+2))

def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['acc'])
    plt.plot(x_plot, network_history.history['val_acc'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('F1')
    plt.plot(x_plot, network_history.history['f1_m'])
    plt.plot(x_plot, network_history.history['val_f1_m'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    
plot_history(history)

In [0]:
test_loss, test_acc, F1 = model.evaluate(X_test, y_test, batch_size=128)
print("TEST LOSS:", test_loss)
print("TEST ACCURACY:", test_acc)
print("TEST F1:", F1)

5400/5400 [==============================] - 0s 9us/step
TEST LOSS: 0.5734567669585899
TEST ACCURACY: 0.7757407406524376
TEST F1: 0.5243786685113554


In [0]:
previsione = model.predict_classes(X_test)
print(classification_report(y_test, previsione))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4205
           1       0.49      0.57      0.53      1195

    accuracy                           0.78      5400
   macro avg       0.68      0.70      0.69      5400
weighted avg       0.79      0.78      0.78      5400



## Classificazione sui dati di Test.csv

In [0]:
test.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,30000.0,2,3,1,44,1,-2,-2,-2,-2,-1,0.0,0.0,0.0,0.0,0.0,379.0,0.0,0.0,0.0,0.0,379.0,0.0
1,260000.0,2,1,2,28,0,0,0,0,0,0,85666.0,92286.0,93742.0,93369.0,82476.0,73789.0,10005.0,4026.0,3481.0,141.0,20074.0,60308.0
2,80000.0,1,1,2,25,0,0,0,0,0,0,65734.0,58048.0,53012.0,53571.0,56721.0,57312.0,3000.0,3404.0,2000.0,4000.0,3000.0,5000.0
3,70000.0,2,2,2,42,1,2,0,0,0,0,70803.0,67344.0,59018.0,48033.0,48833.0,49736.0,10.0,2200.0,2100.0,1900.0,1841.0,1945.0
4,400000.0,1,3,1,33,0,0,2,0,0,0,2487.0,4708.0,2192.0,11111.0,5675.0,5071.0,4009.0,6.0,10248.0,4721.0,4114.0,1855.0


In [0]:
test.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

## Preprocessing sul Test

In [0]:
#AGE
temp =  pd.get_dummies(pd.cut(test["AGE"], [0, 20,30,40,50,70, float("inf")]))
temp.columns = ["0-20", "21-30", "31-40", "41-50", "51-70", "71+"]
test = test.join(temp)
test = test.drop(columns="AGE")

#EDUCATION
# Si decide di accorpare tutti i valori superiori a 3 in quanto sono titoli di studio diversi dai più comuni e quindi rari, saranno indicati con 0
test["EDUCATION"][test["EDUCATION"]>3] =0
temp =  pd.get_dummies(pd.cut(test["EDUCATION"], [0, 1, 2, 3, 4]))
temp.columns = ["0", "1", "2", "3"]
test = test.join(temp)
test = test.drop(columns="EDUCATION")

#MARRIAGE
temp =  pd.get_dummies(pd.cut(test["MARRIAGE"], [0, 1, 2, 3,4]))
temp.columns = ["SPOSATO0", "SPOSATO1", "SPOSATO2", "SPOSATO3"]
test = test.join(temp)
test = test.drop(columns="MARRIAGE")

#LIMIT_BAL
temp =  pd.get_dummies(pd.cut(test["LIMIT_BAL"], [0, 80000, 150000, 230000, 310000, 390000, 470000, 550000, 630000, 710000, 1000001]))
temp.columns = ["LIMITE_1", "LIMITE_2", "LIMITE_3", "LIMITE_4", "LIMITE_5", "LIMITE_6", "LIMITE_7", "LIMITE_8", "LIMITE_9", "LIMITE_10"]
test = test.join(temp)
test = test.drop(columns="LIMIT_BAL")

#BILL_AMT
#Tutti i valori negativi sono convertiti in zero per potervi applicare il logaritmo (non applicabile altrimenti se non 
#con l'ottenimento di numeri complessi)
for i in range(1,7):
    test["BILL_AMT" + str(i)][test["BILL_AMT" + str(i)] < 0] = 0
#Si applica la trasformazione logaritmica log(num + 1)
for i in range(1,7):
    test["BILL_AMT" + str(i)] = np.log(test["BILL_AMT" + str(i)] + 1)
    test["PAY_AMT" + str(i)] = np.log(test["PAY_AMT" + str(i)] + 1)
test["AVG_BILL_AMT"] = test[["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"]].mean(axis = 1)
test = test.drop(columns=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

#DS Intermedio
test = test[['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20', '21-30', '31-40', '41-50',
       '51-70', '71+', '0', '1', '2', '3', 'SPOSATO0', 'SPOSATO1', 'SPOSATO2',
       'SPOSATO3', 'LIMITE_1', 'LIMITE_2', 'LIMITE_3', 'LIMITE_4', 'LIMITE_5',
       'LIMITE_6', 'LIMITE_7', 'LIMITE_8', 'LIMITE_9', 'LIMITE_10',
       'AVG_BILL_AMT']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
test.columns

Index(['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', '0-20',
       '21-30', '31-40', '41-50', '51-70', '71+', '0', '1', '2', '3',
       'SPOSATO0', 'SPOSATO1', 'SPOSATO2', 'SPOSATO3', 'LIMITE_1', 'LIMITE_2',
       'LIMITE_3', 'LIMITE_4', 'LIMITE_5', 'LIMITE_6', 'LIMITE_7', 'LIMITE_8',
       'LIMITE_9', 'LIMITE_10', 'AVG_BILL_AMT'],
      dtype='object')

In [0]:
test = test.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Sistemazione colonne

## Previsione dei valori test

In [0]:
y_test = model.predict_classes(test)

In [0]:
y_test[:5]

array([[1],
       [0],
       [0],
       [1],
       [0]], dtype=int32)

#### Scrittura su file dei risultati per la consegna

In [0]:
risultati = []
for i in y_test:
    risultati.append(str(i[0]))
with open("Paolo_Mariani_800307_score1.txt", "w") as f:
    for i in risultati:
        f.write("%s\n" % str(i))

Controllo che sia scritto correttamente

In [0]:
with open("Paolo_Mariani_800307_score1.txt", "r") as f:
  for i in f:
    a = f.readline()
    print(a)

0

1

1

0

0

0

0

1

0

0

0

0

1

0

0

0

0

0

0

0

1

0

0

0

0

1

0

0

0

0

0

0

0

0

0

1

1

0

0

0

1

0

0

0

0

0

0

0

1

0

1

0

0

1

0

0

1

0

1

1

0

0

0

0

0

1

0

1

0

1

1

1

1

0

1

0

0

0

1

0

0

0

1

0

0

1

1

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

1

0

0

0

1

0

0

0

0

0

1

1

0

0

0

0

0

1

0

0

1

0

0

1

0

1

1

0

1

0

1

0

0

0

0

0

0

0

0

0

0

0

1

1

0

1

1

1

1

0

0

0

0

0

0

0

0

1

0

0

0

1

0

1

0

0

1

0

1

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

1

1

1

0

0

0

0

0

1

0

0

0

1

1

1

0

1

0

1

0

1

0

0

1

1

1

0

0

0

0

0

0

1

0

1

0

1

1

0

1

0

0

0

1

0

0

0

1

1

0

0

1

1

1

0

0

0

0

0

0

0

1

0

0

0

0

1

0

0

0

0

1

0

1

1

0

0

1

0

0

1

0

0

1

0

0

0

0

0

0

1

0

0

0

1

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

1

1

0

0

0

0

0

0

0

1

1

0

1

0

0

0

0

0

1

0

0

0

0

0

0

0

1

0

1

1

1

0

0

1